## MIG Configuration Workflow (Admin)

```bash
# Step 1: Enable MIG mode (requires GPU reset)
sudo nvidia-smi -i 0 -mig 1
sudo nvidia-smi --gpu-reset

# Step 2: List available profiles
nvidia-smi mig -lgip

# Step 3: Create GPU instances
# Example: Create two 3g.40gb instances on H100
sudo nvidia-smi mig -cgi 9,9 -i 0

# Step 4: Create compute instances for each GPU instance
sudo nvidia-smi mig -cci -gi 0
sudo nvidia-smi mig -cci -gi 1

# Step 5: List created instances
nvidia-smi mig -lgi
nvidia-smi mig -lci
```

## Query MIG Configuration with NVML

In [ ]:
%%writefile mig_config_query.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <nvml.h>

#define CHECK_NVML(call) \
    do { \
        nvmlReturn_t result = call; \
        if (result != NVML_SUCCESS) { \
            printf("NVML: %s\n", nvmlErrorString(result)); \
            return; \
        } \
    } while(0)

void queryMIGConfiguration(unsigned int deviceIdx) {
    nvmlDevice_t device;
    CHECK_NVML(nvmlDeviceGetHandleByIndex(deviceIdx, &device));
    
    char name[NVML_DEVICE_NAME_BUFFER_SIZE];
    CHECK_NVML(nvmlDeviceGetName(device, name, sizeof(name)));
    printf("\n=== Device %u: %s ===\n", deviceIdx, name);
    
    // Check MIG mode
    unsigned int currentMode, pendingMode;
    nvmlReturn_t migResult = nvmlDeviceGetMigMode(device, &currentMode, &pendingMode);
    
    if (migResult == NVML_ERROR_NOT_SUPPORTED) {
        printf("MIG not supported on this GPU\n");
        return;
    }
    CHECK_NVML(migResult);
    
    printf("MIG Mode: %s (pending: %s)\n",
           currentMode ? "ENABLED" : "DISABLED",
           pendingMode ? "ENABLED" : "DISABLED");
    
    if (!currentMode) {
        printf("MIG is disabled. Enable with: sudo nvidia-smi -mig 1\n");
        return;
    }
    
    // List GPU instance profiles
    printf("\nAvailable GPU Instance Profiles:\n");
    for (unsigned int profileId = 0; profileId < 20; profileId++) {
        nvmlGpuInstanceProfileInfo_t profileInfo;
        nvmlReturn_t result = nvmlDeviceGetGpuInstanceProfileInfo(
            device, profileId, &profileInfo);
        
        if (result == NVML_SUCCESS) {
            printf("  Profile %u: memory=%llu MB, slices=%u, instances=%u\n",
                   profileId,
                   profileInfo.memorySizeMB,
                   profileInfo.sliceCount,
                   profileInfo.instanceCount);
        }
    }
    
    // List existing GPU instances
    printf("\nExisting GPU Instances:\n");
    unsigned int count = 0;
    nvmlGpuInstance_t gpuInstances[NVML_MAX_GPU_INSTANCES];
    
    // Try to get instances for each profile
    for (unsigned int profileId = 0; profileId < 20; profileId++) {
        nvmlGpuInstanceProfileInfo_t profileInfo;
        if (nvmlDeviceGetGpuInstanceProfileInfo(device, profileId, &profileInfo) == NVML_SUCCESS) {
            unsigned int instCount;
            if (nvmlDeviceGetGpuInstances(device, profileId, gpuInstances, &instCount) == NVML_SUCCESS) {
                for (unsigned int i = 0; i < instCount; i++) {
                    nvmlGpuInstanceInfo_t info;
                    if (nvmlGpuInstanceGetInfo(gpuInstances[i], &info) == NVML_SUCCESS) {
                        printf("  Instance %u: id=%u, profileId=%u\n",
                               count++, info.id, info.profileId);
                    }
                }
            }
        }
    }
    
    if (count == 0) {
        printf("  No GPU instances created\n");
    }
}

int main() {
    printf("=== MIG Configuration Query ===\n");
    
    nvmlReturn_t result = nvmlInit();
    if (result != NVML_SUCCESS) {
        printf("Failed to initialize NVML: %s\n", nvmlErrorString(result));
        return 1;
    }
    
    unsigned int deviceCount;
    CHECK_NVML(nvmlDeviceGetCount(&deviceCount));
    
    for (unsigned int i = 0; i < deviceCount; i++) {
        queryMIGConfiguration(i);
    }
    
    nvmlShutdown();
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 mig_config_query.cu -o mig_config_query -lnvidia-ml && ./mig_config_query

## Targeting Specific MIG Instances

When MIG is enabled, each instance has a unique UUID:

In [ ]:
# List MIG devices with UUIDs
!nvidia-smi -L 2>/dev/null

In [ ]:
%%writefile target_mig_instance.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

__global__ void simpleKernel(int* result) {
    *result = blockIdx.x * blockDim.x + threadIdx.x;
}

int main(int argc, char** argv) {
    printf("=== Target MIG Instance Demo ===\n\n");
    
    // Check environment
    const char* cvd = getenv("CUDA_VISIBLE_DEVICES");
    printf("CUDA_VISIBLE_DEVICES: %s\n", cvd ? cvd : "(not set)");
    
    int deviceCount;
    CHECK_CUDA(cudaGetDeviceCount(&deviceCount));
    printf("Available CUDA devices: %d\n\n", deviceCount);
    
    for (int i = 0; i < deviceCount; i++) {
        cudaDeviceProp prop;
        CHECK_CUDA(cudaGetDeviceProperties(&prop, i));
        
        printf("Device %d: %s\n", i, prop.name);
        printf("  Memory: %.2f GB\n", 
               prop.totalGlobalMem / (1024.0*1024.0*1024.0));
        printf("  SMs: %d\n", prop.multiProcessorCount);
        
        // Set device and run kernel
        CHECK_CUDA(cudaSetDevice(i));
        
        int* d_result;
        int h_result = -1;
        CHECK_CUDA(cudaMalloc(&d_result, sizeof(int)));
        
        simpleKernel<<<1, 1>>>(d_result);
        CHECK_CUDA(cudaDeviceSynchronize());
        CHECK_CUDA(cudaMemcpy(&h_result, d_result, sizeof(int), 
                               cudaMemcpyDeviceToHost));
        
        printf("  Kernel result: %d\n", h_result);
        CHECK_CUDA(cudaFree(d_result));
    }
    
    printf("\nTo target a specific MIG instance:\n");
    printf("  export CUDA_VISIBLE_DEVICES=MIG-<uuid>\n");
    printf("  ./program\n");
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 target_mig_instance.cu -o target_mig_instance && ./target_mig_instance

## MIG with CUDA MPS

MPS (Multi-Process Service) can run on MIG instances for even finer sharing:

```bash
# Start MPS on a specific MIG instance
export CUDA_VISIBLE_DEVICES=MIG-<uuid>
export CUDA_MPS_PIPE_DIRECTORY=/tmp/nvidia-mps-mig
export CUDA_MPS_LOG_DIRECTORY=/tmp/nvidia-mps-log

nvidia-cuda-mps-control -d

# Run multiple processes sharing the MIG instance
./app1 &
./app2 &
./app3 &
wait

# Stop MPS
echo quit | nvidia-cuda-mps-control
```

## Compute Instance Profiles

Each GPU Instance can be further divided into Compute Instances:

In [ ]:
%%writefile compute_instance_query.cu
#include <stdio.h>
#include <nvml.h>

#define CHECK_NVML(call) \
    do { \
        nvmlReturn_t result = call; \
        if (result != NVML_SUCCESS) { \
            printf("NVML: %s\n", nvmlErrorString(result)); \
            return; \
        } \
    } while(0)

void listComputeInstanceProfiles(nvmlGpuInstance_t gpuInstance, unsigned int giId) {
    printf("  Compute Instance Profiles for GPU Instance %u:\n", giId);
    
    for (unsigned int profileId = 0; profileId < 10; profileId++) {
        nvmlComputeInstanceProfileInfo_t profileInfo;
        nvmlReturn_t result = nvmlGpuInstanceGetComputeInstanceProfileInfo(
            gpuInstance, profileId, NVML_COMPUTE_INSTANCE_ENGINE_PROFILE_SHARED,
            &profileInfo);
        
        if (result == NVML_SUCCESS) {
            printf("    Profile %u: SMs=%u, instances=%u\n",
                   profileId,
                   profileInfo.smCount,
                   profileInfo.instanceCount);
        }
    }
}

int main() {
    printf("=== Compute Instance Profiles ===\n\n");
    
    if (nvmlInit() != NVML_SUCCESS) {
        printf("Failed to initialize NVML\n");
        return 1;
    }
    
    nvmlDevice_t device;
    CHECK_NVML(nvmlDeviceGetHandleByIndex(0, &device));
    
    unsigned int currentMode, pendingMode;
    if (nvmlDeviceGetMigMode(device, &currentMode, &pendingMode) != NVML_SUCCESS ||
        !currentMode) {
        printf("MIG not enabled. Showing theoretical profiles...\n\n");
        
        // Show what would be available
        printf("For 3g.40gb GPU Instance:\n");
        printf("  - 3c.3g.40gb: All 3 slices (full)\n");
        printf("  - 2c.3g.40gb: 2/3 compute slices\n");
        printf("  - 1c.3g.40gb: 1/3 compute slices\n");
        printf("\nFor 7g.80gb GPU Instance (A100/H100):\n");
        printf("  - 7c.7g.80gb: All compute (full)\n");
        printf("  - 4c.7g.80gb: 4/7 compute\n");
        printf("  - 3c.7g.80gb: 3/7 compute\n");
        printf("  - 2c.7g.80gb: 2/7 compute\n");
        printf("  - 1c.7g.80gb: 1/7 compute\n");
    } else {
        // Query actual instances
        printf("MIG enabled. Querying actual configuration...\n\n");
        
        nvmlGpuInstance_t gpuInstances[8];
        for (unsigned int profileId = 0; profileId < 20; profileId++) {
            unsigned int count;
            if (nvmlDeviceGetGpuInstances(device, profileId, gpuInstances, &count) == NVML_SUCCESS) {
                for (unsigned int i = 0; i < count; i++) {
                    nvmlGpuInstanceInfo_t info;
                    if (nvmlGpuInstanceGetInfo(gpuInstances[i], &info) == NVML_SUCCESS) {
                        listComputeInstanceProfiles(gpuInstances[i], info.id);
                    }
                }
            }
        }
    }
    
    nvmlShutdown();
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 compute_instance_query.cu -o compute_instance_query -lnvidia-ml && ./compute_instance_query

## MIG Configuration Reference

### H100 80GB MIG Configurations

| Configuration | GPU Instances | Memory per Instance |
|--------------|---------------|---------------------|
| 7 x 1g.10gb | 7 | 10GB each |
| 3 x 2g.20gb + 1g.10gb | 4 | 20GB, 20GB, 20GB, 10GB |
| 2 x 3g.40gb | 2 | 40GB each |
| 1 x 4g.40gb + 1 x 3g.40gb | 2 | 40GB, 40GB |
| 1 x 7g.80gb | 1 | 80GB (full) |

### A100 80GB MIG Configurations

| Configuration | GPU Instances |
|--------------|---------------|
| 7 x 1g.10gb | 7 instances |
| 3 x 2g.20gb + 1g.10gb | 4 instances |
| 2 x 3g.40gb | 2 instances |
| 1 x 7g.80gb | Full GPU |

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice MIG configuration concepts:

1. **Exercise 1: Instance-Aware Resource Management** - Allocate resources based on MIG instance capabilities
2. **Exercise 2: Multi-Instance Coordination** - Design patterns for running across MIG instances
3. **Exercise 3: Configuration Validation** - Verify and report MIG configuration programmatically

In [ ]:
%%writefile mig_configuration_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t error = call; \
        if (error != cudaSuccess) { \
            fprintf(stderr, "CUDA Error: %s at %s:%d\n", \
                    cudaGetErrorString(error), __FILE__, __LINE__); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Instance-Aware Resource Management
// Allocate resources based on MIG instance capabilities
// =============================================================================

struct MIGResourceProfile {
    size_t maxMemory;
    int maxSMs;
    int recommendedBlockSize;
    int recommendedGridSize;
    float memoryReserveFactor;
};

MIGResourceProfile detectResourceProfile() {
    MIGResourceProfile profile;
    
    cudaDeviceProp prop;
    CUDA_CHECK(cudaGetDeviceProperties(&prop, 0));
    
    size_t freeMemory, totalMemory;
    CUDA_CHECK(cudaMemGetInfo(&freeMemory, &totalMemory));
    
    profile.maxMemory = totalMemory;
    profile.maxSMs = prop.multiProcessorCount;
    
    // Classify instance size based on SM count (approximate)
    if (prop.multiProcessorCount <= 14) {
        // Small MIG instance (1g.10gb equivalent)
        profile.recommendedBlockSize = 128;
        profile.recommendedGridSize = prop.multiProcessorCount * 4;
        profile.memoryReserveFactor = 0.6f;
    } else if (prop.multiProcessorCount <= 28) {
        // Medium MIG instance (2g.20gb equivalent)
        profile.recommendedBlockSize = 256;
        profile.recommendedGridSize = prop.multiProcessorCount * 8;
        profile.memoryReserveFactor = 0.7f;
    } else if (prop.multiProcessorCount <= 56) {
        // Large MIG instance (3g.40gb or 4g.40gb)
        profile.recommendedBlockSize = 256;
        profile.recommendedGridSize = prop.multiProcessorCount * 12;
        profile.memoryReserveFactor = 0.75f;
    } else {
        // Full GPU or 7g.80gb
        profile.recommendedBlockSize = 256;
        profile.recommendedGridSize = prop.multiProcessorCount * 16;
        profile.memoryReserveFactor = 0.8f;
    }
    
    return profile;
}

void exercise1_resource_management() {
    printf("=== Exercise 1: Instance-Aware Resource Management ===\n\n");
    
    cudaDeviceProp prop;
    CUDA_CHECK(cudaGetDeviceProperties(&prop, 0));
    
    MIGResourceProfile profile = detectResourceProfile();
    
    printf("Device: %s\n", prop.name);
    printf("Detected Profile:\n");
    printf("  Max Memory: %.2f GB\n", profile.maxMemory / (1024.0 * 1024.0 * 1024.0));
    printf("  Available SMs: %d\n", profile.maxSMs);
    printf("  Recommended Block Size: %d\n", profile.recommendedBlockSize);
    printf("  Recommended Grid Size: %d\n", profile.recommendedGridSize);
    printf("  Memory Reserve Factor: %.0f%%\n", profile.memoryReserveFactor * 100);
    
    // Calculate safe allocation
    size_t safeAlloc = (size_t)(profile.maxMemory * profile.memoryReserveFactor * 0.5);
    printf("\nSafe allocation for workload: %.2f GB\n\n", 
           safeAlloc / (1024.0 * 1024.0 * 1024.0));
}

// =============================================================================
// Exercise 2: Multi-Instance Coordination
// Design patterns for running across MIG instances
// =============================================================================

__global__ void workloadKernel(float* data, int n, int instanceId) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    for (int i = idx; i < n; i += stride) {
        // Simulate work with instance-specific computation
        data[i] = sinf((float)i + instanceId * 1000.0f);
    }
}

void exercise2_multi_instance_coordination() {
    printf("=== Exercise 2: Multi-Instance Coordination ===\n\n");
    
    // In a real MIG setup, each instance would be a separate device
    // Here we simulate the pattern for a single device
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    printf("Available CUDA devices (or MIG instances): %d\n\n", deviceCount);
    
    // Pattern: Work distribution across instances
    printf("Multi-Instance Work Distribution Pattern:\n\n");
    
    for (int dev = 0; dev < deviceCount; dev++) {
        CUDA_CHECK(cudaSetDevice(dev));
        
        cudaDeviceProp prop;
        CUDA_CHECK(cudaGetDeviceProperties(&prop, dev));
        
        size_t freeMemory, totalMemory;
        CUDA_CHECK(cudaMemGetInfo(&freeMemory, &totalMemory));
        
        printf("Instance %d: %s\n", dev, prop.name);
        printf("  SMs: %d, Memory: %.2f GB\n", 
               prop.multiProcessorCount, 
               totalMemory / (1024.0 * 1024.0 * 1024.0));
        
        // Calculate proportional workload based on SM count
        // (In multi-MIG setup, different instances may have different sizes)
        int workloadSize = prop.multiProcessorCount * 1024 * 256;
        printf("  Assigned workload: %d elements\n\n", workloadSize);
        
        // Allocate and process
        float* d_data;
        CUDA_CHECK(cudaMalloc(&d_data, workloadSize * sizeof(float)));
        
        int blockSize = 256;
        int gridSize = (workloadSize + blockSize - 1) / blockSize;
        gridSize = min(gridSize, prop.multiProcessorCount * 8);
        
        workloadKernel<<<gridSize, blockSize>>>(d_data, workloadSize, dev);
        CUDA_CHECK(cudaDeviceSynchronize());
        
        printf("  Workload completed on instance %d\n\n", dev);
        
        CUDA_CHECK(cudaFree(d_data));
    }
    
    CUDA_CHECK(cudaSetDevice(0));  // Reset to device 0
}

// =============================================================================
// Exercise 3: Configuration Validation
// Verify and report MIG configuration programmatically
// =============================================================================

struct DeviceCapabilityReport {
    char name[256];
    int smCount;
    size_t totalMemory;
    size_t freeMemory;
    int computeCapabilityMajor;
    int computeCapabilityMinor;
    bool isMIGCapable;
    bool isLikelyMIGInstance;
};

DeviceCapabilityReport validateConfiguration(int deviceId) {
    DeviceCapabilityReport report;
    
    cudaDeviceProp prop;
    CUDA_CHECK(cudaGetDeviceProperties(&prop, deviceId));
    
    strncpy(report.name, prop.name, 255);
    report.name[255] = '\0';
    report.smCount = prop.multiProcessorCount;
    report.computeCapabilityMajor = prop.major;
    report.computeCapabilityMinor = prop.minor;
    
    CUDA_CHECK(cudaSetDevice(deviceId));
    CUDA_CHECK(cudaMemGetInfo(&report.freeMemory, &report.totalMemory));
    
    // MIG is available on A100 (8.0), H100 (9.0), and newer datacenter GPUs
    report.isMIGCapable = (prop.major >= 8);
    
    // Heuristic: MIG instances typically have reduced SM count
    // A100 has 108 SMs, H100 has 132 SMs - MIG instances have less
    report.isLikelyMIGInstance = (prop.major >= 8 && prop.multiProcessorCount < 60);
    
    return report;
}

void exercise3_configuration_validation() {
    printf("=== Exercise 3: Configuration Validation ===\n\n");
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    printf("╔═══════════════════════════════════════════════════════════════════╗\n");
    printf("║                    MIG CONFIGURATION REPORT                       ║\n");
    printf("╠═══════════════════════════════════════════════════════════════════╣\n");
    
    for (int dev = 0; dev < deviceCount; dev++) {
        DeviceCapabilityReport report = validateConfiguration(dev);
        
        printf("║ Device %d: %-55s ║\n", dev, report.name);
        printf("║   Compute Capability: %d.%d                                       ║\n",
               report.computeCapabilityMajor, report.computeCapabilityMinor);
        printf("║   SM Count: %-3d                                                  ║\n",
               report.smCount);
        printf("║   Total Memory: %6.2f GB                                        ║\n",
               report.totalMemory / (1024.0 * 1024.0 * 1024.0));
        printf("║   Free Memory:  %6.2f GB                                        ║\n",
               report.freeMemory / (1024.0 * 1024.0 * 1024.0));
        printf("║   MIG Capable: %-3s                                               ║\n",
               report.isMIGCapable ? "Yes" : "No");
        printf("║   Likely MIG Instance: %-3s                                       ║\n",
               report.isLikelyMIGInstance ? "Yes" : "No");
        
        // Provide recommendations
        printf("║                                                                   ║\n");
        printf("║   Recommendations:                                                ║\n");
        
        if (report.isLikelyMIGInstance) {
            printf("║   - Use adaptive memory allocation (check free memory first)     ║\n");
            printf("║   - Limit grid size to %d blocks for optimal performance        ║\n",
                   report.smCount * 8);
        } else {
            printf("║   - Full GPU resources available                                 ║\n");
            printf("║   - Consider enabling MIG for multi-tenant workloads             ║\n");
        }
        printf("╠═══════════════════════════════════════════════════════════════════╣\n");
    }
    printf("╚═══════════════════════════════════════════════════════════════════╝\n\n");
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║    Week 18 Day 2: MIG Configuration - CUDA C++ Exercises     ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    exercise1_resource_management();
    exercise2_multi_instance_coordination();
    exercise3_configuration_validation();
    
    printf("All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_75 -o mig_configuration_exercises mig_configuration_exercises.cu && ./mig_configuration_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Python with PyCUDA/Numba for those who prefer Python-based GPU programming. MIG configuration concepts translate well to Python using subprocess calls to nvidia-smi or NVML Python bindings.

**Python Exercise Ideas:**
1. Use `py3nvml` to query MIG configuration and GPU/Compute instances
2. Implement a Python context manager for MIG-aware device selection
3. Create a workload scheduler that distributes tasks across MIG instances

## Key Takeaways

1. **GPU Instance** - Memory and SM partition
2. **Compute Instance** - Further subdivides SMs within a GPU Instance
3. **Use CUDA_VISIBLE_DEVICES** - Target specific MIG instances via UUID
4. **NVML API** - Query MIG configuration programmatically
5. **Admin required** - MIG configuration needs root access